In [1]:
# Required Modules
import pandas as pd
import numpy as np
import lazypredict
import warnings
warnings.filterwarnings("ignore")

from scipy import stats

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from math import sqrt


from lazypredict.Supervised import LazyRegressor
from lazypredict.Supervised import LazyClassifier


In [2]:
data = pd.read_csv('Resources/Clean_Data/Final_Data.csv')
data.head()

,Zip_Code,Date,Date_Code,Year,Month,Sale_Price,Interest_Rate,Property_Tax,Rent_Price,Household_Income,Rent_Affordability,Total_Sales,FHA_Count,Mobility_Rate,School_Rating,School_Grade,Owner_Occupied,Renter_Occupied,Total_Vacant,Total_Dwellings,FTE_Employed,Unemployed,Expense_Index,Average_Commute,Crime_Index,Loan_Amount,Loan_Term,Loan_R,Loan_Payment,Home_Affordability
0,32003,2019-01-01,201901,2019,1,244950.00,4.46,3778,1113,94154.00,0.14,32,7,0.18,0,Unknown,8834,2186,704,11724,10671,1069,104,35,28,195960.00,360.00,1.00,988.71,0.13
1,32003,2019-02-01,201902,2019,2,270000.00,4.37,3778,1113,94154.00,0.14,41,2,0.18,0,Unknown,8834,2186,704,11724,10671,1069,104,35,28,216000.00,360.00,1.00,1077.82,0.14
2,32003,2019-03-01,201903,2019,3,275000.00,4.26,3778,1113,94154.00,0.14,21,7,0.18,0,Unknown,8834,2186,704,11724,10671,1069,104,35,28,220000.00,360.00,1.00,1084.20,0.14
3,32003,2019-04-01,201904,2019,4,264500.00,4.14,3778,1113,94154.00,0.14,42,3,0.18,0,Unknown,8834,2186,704,11724,10671,1069,104,35,28,211600.00,360.00,1.00,1027.67,0.13
4,32003,2019-05-01,201905,2019,5,281000.00,4.07,3778,1113,94154.00,0.14,66,8,0.18,0,Unknown,8834,2186,704,11724,10671,1069,104,35,28,224800.00,360.00,1.00,1082.58,0.14


In [24]:
ml_data = data[['Zip_Code', 'Year', 'Month', 'Mobility_Rate', 'Expense_Index', 'Crime_Index' , 'Total_Vacant', 'Total_Dwellings', 'Total_Sales', 'FHA_Count', 'Home_Affordability', 'Rent_Affordability', 'Sale_Price' ]]
# ml_data.index = ml_data['Zip_Code']
ml_data.head()

,Zip_Code,Year,Month,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price
0,32003,2019,1,0.18,104,28,704,11724,32,7,0.13,0.14,244950.00
1,32003,2019,2,0.18,104,28,704,11724,41,2,0.14,0.14,270000.00
2,32003,2019,3,0.18,104,28,704,11724,21,7,0.14,0.14,275000.00
3,32003,2019,4,0.18,104,28,704,11724,42,3,0.13,0.14,264500.00
4,32003,2019,5,0.18,104,28,704,11724,66,8,0.14,0.14,281000.00


In [25]:
ml_data_3mo = ml_data.sort_values(by = ['Zip_Code', 'Year', 'Month'], ascending = [True, True, True])
ml_data_3mo.head()

,Zip_Code,Year,Month,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price
0,32003,2019,1,0.18,104,28,704,11724,32,7,0.13,0.14,244950.00
1,32003,2019,2,0.18,104,28,704,11724,41,2,0.14,0.14,270000.00
2,32003,2019,3,0.18,104,28,704,11724,21,7,0.14,0.14,275000.00
3,32003,2019,4,0.18,104,28,704,11724,42,3,0.13,0.14,264500.00
4,32003,2019,5,0.18,104,28,704,11724,66,8,0.14,0.14,281000.00


In [26]:
ml_data_3mo['Last_Month_Price'] = ml_data_3mo.groupby(['Zip_Code'])['Sale_Price'].shift()
ml_data_3mo['Last_2Month_Price'] = ml_data_3mo.groupby(['Zip_Code'])['Sale_Price'].shift(2)
ml_data_3mo['Last_3Month_Price'] = ml_data_3mo.groupby(['Zip_Code'])['Sale_Price'].shift(3)


price_difference_1 = []
price_difference_2 = []
price_difference_3 = []


for column in ml_data_3mo[['Sale_Price', 'Last_Month_Price', 'Last_2Month_Price',
                    'Last_3Month_Price']]:
   # Select column contents by column name using [] operator
    current_sales = ml_data_3mo['Sale_Price'].values
    last_sales = ml_data_3mo['Last_Month_Price'].values
    last_2mo_sale = ml_data_3mo['Last_2Month_Price'].values
    last_3mo_sale = ml_data_3mo['Last_3Month_Price'].values
    price_difference_1.append(current_sales-last_sales)
    price_difference_2.append(last_sales-last_2mo_sale)
    price_difference_3.append(last_2mo_sale-last_3mo_sale)
    

ml_data_3mo['Last_Month_Diff'] = price_difference_1[0]
ml_data_3mo['Last_2Month_Diff'] = price_difference_2[0]
ml_data_3mo['Last_3Month_Diff'] = price_difference_3[0]

ml_data_3mo.head()

,Zip_Code,Year,Month,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price,Last_Month_Price,Last_2Month_Price,Last_3Month_Price,Last_Month_Diff,Last_2Month_Diff,Last_3Month_Diff
0,32003,2019,1,0.18,104,28,704,11724,32,7,0.13,0.14,244950.00,nan,nan,nan,nan,nan,nan
1,32003,2019,2,0.18,104,28,704,11724,41,2,0.14,0.14,270000.00,244950.00,nan,nan,25050.00,nan,nan
2,32003,2019,3,0.18,104,28,704,11724,21,7,0.14,0.14,275000.00,270000.00,244950.00,nan,5000.00,25050.00,nan
3,32003,2019,4,0.18,104,28,704,11724,42,3,0.13,0.14,264500.00,275000.00,270000.00,244950.00,-10500.00,5000.00,25050.00
4,32003,2019,5,0.18,104,28,704,11724,66,8,0.14,0.14,281000.00,264500.00,275000.00,270000.00,16500.00,-10500.00,5000.00


In [8]:
ml_data_3mo.replace([np.inf, -np.inf], np.nan, inplace=True)
ml_data_3mo.dropna(inplace=True)
ml_data_3mo.head()

,Zip_Code,Year,Month,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price,Last_Month_Price,Last_2Month_Price,Last_3Month_Price,Last_Month_Diff,Last_2Month_Diff,Last_3Month_Diff
3,32003,2019,4,0.18,104,28,704,11724,42,3,0.13,0.14,264500.00,275000.00,270000.00,244950.00,-10500.00,5000.00,25050.00
4,32003,2019,5,0.18,104,28,704,11724,66,8,0.14,0.14,281000.00,264500.00,275000.00,270000.00,16500.00,-10500.00,5000.00
5,32003,2019,6,0.18,104,28,704,11724,43,3,0.13,0.14,275000.00,281000.00,264500.00,275000.00,-6000.00,16500.00,-10500.00
6,32003,2019,7,0.18,104,28,704,11724,90,7,0.13,0.14,282500.00,275000.00,281000.00,264500.00,7500.00,-6000.00,16500.00
7,32003,2019,8,0.18,104,28,704,11724,64,5,0.13,0.14,269577.00,282500.00,275000.00,281000.00,-12923.00,7500.00,-6000.00


In [10]:
y_value = ml_data_3mo['Sale_Price'].values

x_values = ml_data_3mo.drop(columns='Sale_Price')

feature_names = x_values.columns

x_values

,Zip_Code,Year,Month,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Last_Month_Price,Last_2Month_Price,Last_3Month_Price,Last_Month_Diff,Last_2Month_Diff,Last_3Month_Diff
3,32003,2019,4,0.18,104,28,704,11724,42,3,0.13,0.14,275000.00,270000.00,244950.00,-10500.00,5000.00,25050.00
4,32003,2019,5,0.18,104,28,704,11724,66,8,0.14,0.14,264500.00,275000.00,270000.00,16500.00,-10500.00,5000.00
5,32003,2019,6,0.18,104,28,704,11724,43,3,0.13,0.14,281000.00,264500.00,275000.00,-6000.00,16500.00,-10500.00
6,32003,2019,7,0.18,104,28,704,11724,90,7,0.13,0.14,275000.00,281000.00,264500.00,7500.00,-6000.00,16500.00
7,32003,2019,8,0.18,104,28,704,11724,64,5,0.13,0.14,282500.00,275000.00,281000.00,-12923.00,7500.00,-6000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23505,34997,2020,11,0.14,91,113,2976,21551,132,11,0.17,0.25,285000.00,256000.00,251000.00,-25250.00,29000.00,5000.00
23506,34997,2020,12,0.14,91,113,2976,21551,142,16,0.20,0.25,259750.00,285000.00,256000.00,37250.00,-25250.00,29000.00
23507,34997,2021,1,0.14,91,113,2976,21551,81,10,0.20,0.25,297000.00,259750.00,285000.00,8000.00,37250.00,-25250.00
23508,34997,2021,2,0.14,91,113,2976,21551,116,7,0.20,0.25,305000.00,297000.00,259750.00,-5000.00,8000.00,37250.00


In [11]:
X_train, X_test, y_train, y_test = train_test_split(x_values, y_value, random_state=42)

In [12]:
reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)
reg_models, reg_predictions = reg.fit(X_train, X_test, y_train, y_test)

print(reg_models)

100%|██████████| 42/42 [01:49<00:00,  2.60s/it]
                               Adjusted R-Squared  ...  Time Taken
Model                                              ...            
OrthogonalMatchingPursuitCV                  1.00  ...        0.06
HuberRegressor                               1.00  ...        0.45
TransformedTargetRegressor                   1.00  ...        0.02
LarsCV                                       1.00  ...        0.06
Lars                                         1.00  ...        0.03
RANSACRegressor                              1.00  ...        0.03
LassoLarsCV                                  1.00  ...        0.07
LassoLarsIC                                  1.00  ...        0.02
LinearRegression                             1.00  ...        0.03
PassiveAggressiveRegressor                   1.00  ...        0.11
RidgeCV                                      1.00  ...        0.06
Ridge                                        1.00  ...        0.02
SGDRegressor  

In [27]:
price_change_1mo = []

for row in ml_data_3mo['Last_Month_Diff']:
    if row > 0:
        price_change_1mo.append(1)
    else:
        price_change_1mo.append(0)
ml_data_3mo['Price_Change_1mo'] = price_change_1mo

ml_data_3mo

,Zip_Code,Year,Month,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price,Last_Month_Price,Last_2Month_Price,Last_3Month_Price,Last_Month_Diff,Last_2Month_Diff,Last_3Month_Diff,Price_Change_1mo
0,32003,2019,1,0.18,104,28,704,11724,32,7,0.13,0.14,244950.00,nan,nan,nan,nan,nan,nan,0
1,32003,2019,2,0.18,104,28,704,11724,41,2,0.14,0.14,270000.00,244950.00,nan,nan,25050.00,nan,nan,1
2,32003,2019,3,0.18,104,28,704,11724,21,7,0.14,0.14,275000.00,270000.00,244950.00,nan,5000.00,25050.00,nan,1
3,32003,2019,4,0.18,104,28,704,11724,42,3,0.13,0.14,264500.00,275000.00,270000.00,244950.00,-10500.00,5000.00,25050.00,0
4,32003,2019,5,0.18,104,28,704,11724,66,8,0.14,0.14,281000.00,264500.00,275000.00,270000.00,16500.00,-10500.00,5000.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23505,34997,2020,11,0.14,91,113,2976,21551,132,11,0.17,0.25,259750.00,285000.00,256000.00,251000.00,-25250.00,29000.00,5000.00,0
23506,34997,2020,12,0.14,91,113,2976,21551,142,16,0.20,0.25,297000.00,259750.00,285000.00,256000.00,37250.00,-25250.00,29000.00,1
23507,34997,2021,1,0.14,91,113,2976,21551,81,10,0.20,0.25,305000.00,297000.00,259750.00,285000.00,8000.00,37250.00,-25250.00,1
23508,34997,2021,2,0.14,91,113,2976,21551,116,7,0.20,0.25,300000.00,305000.00,297000.00,259750.00,-5000.00,8000.00,37250.00,0


In [28]:
ml_data_3mo = ml_data_3mo.drop(columns=['Zip_Code', 'Year', 'Month','Last_Month_Price', 'Last_Month_Diff', 'Last_2Month_Price', 'Last_2Month_Diff', 'Last_3Month_Price', 'Last_3Month_Diff'])
ml_data_3mo.head()

,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price,Price_Change_1mo
0,0.18,104,28,704,11724,32,7,0.13,0.14,244950.00,0
1,0.18,104,28,704,11724,41,2,0.14,0.14,270000.00,1
2,0.18,104,28,704,11724,21,7,0.14,0.14,275000.00,1
3,0.18,104,28,704,11724,42,3,0.13,0.14,264500.00,0
4,0.18,104,28,704,11724,66,8,0.14,0.14,281000.00,1


In [29]:
ml_data_3mo.replace([np.inf, -np.inf], np.nan, inplace=True)
ml_data_3mo.dropna(inplace=True)

In [30]:
y_value = ml_data_3mo['Price_Change_1mo'].values

x_values = ml_data_3mo.drop(columns='Price_Change_1mo')

feature_names = x_values.columns

x_values

,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price
0,0.18,104,28,704,11724,32,7,0.13,0.14,244950.00
1,0.18,104,28,704,11724,41,2,0.14,0.14,270000.00
2,0.18,104,28,704,11724,21,7,0.14,0.14,275000.00
3,0.18,104,28,704,11724,42,3,0.13,0.14,264500.00
4,0.18,104,28,704,11724,66,8,0.14,0.14,281000.00
...,...,...,...,...,...,...,...,...,...,...
23505,0.14,91,113,2976,21551,132,11,0.17,0.25,259750.00
23506,0.14,91,113,2976,21551,142,16,0.20,0.25,297000.00
23507,0.14,91,113,2976,21551,81,10,0.20,0.25,305000.00
23508,0.14,91,113,2976,21551,116,7,0.20,0.25,300000.00


In [32]:
X_train, X_test, y_train, y_test = train_test_split(x_values, y_value, random_state=42)

In [33]:
from lazypredict.Supervised import LazyClassifier

clf = LazyClassifier(verbose=0,ignore_warnings=False)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
models

 21%|██        | 6/29 [00:05<00:19,  1.19it/s]CategoricalNB model failed to execute
Negative values in data passed to CategoricalNB (input X)
 90%|████████▉ | 26/29 [01:40<00:13,  4.65s/it]StackingClassifier model failed to execute
__init__() missing 1 required positional argument: 'estimators'
100%|██████████| 29/29 [01:41<00:00,  3.50s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
RandomForestClassifier,0.66,0.66,0.66,0.66,2.66
ExtraTreesClassifier,0.66,0.66,0.66,0.66,1.49
XGBClassifier,0.65,0.65,0.65,0.65,0.51
LGBMClassifier,0.64,0.64,0.64,0.64,0.18
BaggingClassifier,0.64,0.64,0.64,0.64,0.74
NuSVC,0.63,0.63,0.63,0.63,59.86
SVC,0.63,0.63,0.63,0.63,8.59
CalibratedClassifierCV,0.62,0.61,0.61,0.61,4.02
LogisticRegression,0.61,0.61,0.61,0.61,0.05
